In [1]:
from lib.dataset.mesh_util import *
from lib.dataset.TestDataset import TestDataset
from lib.common.config import cfg
from lib.pymafx.core import path_config
import os.path as osp

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
SMPLX_object = SMPLX()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

Type of each parameter in 'dataset_param':

    <class 'str'>
    <class 'NoneType'>
    <class 'bool'>
    <class 'str'>
    <class 'int'>
    <class 'bool'>

In [4]:
dataset_param = {
        "image_dir": './examples/testimg/',
        "seg_dir": None,
        "use_seg": True,    # w/ or w/o segmentation
        "hps_type": 'pymafx' ,   # pymafx/pixie
        "vol_res": cfg.vol_res,
        "single": False, #args.multi,
    }


In [5]:
path_config.pymafx_data_dir = "./data/HPS/pymafx_data"

os.path.exists(path_config.pymafx_data_dir)

True

In [6]:
dataset = TestDataset(dataset_param, device)

AttributeError: MODEL